In [1]:
import tensorflow as tf

from local_tabnet import TabNet as LocalTabNet
from google_tabnet import TabNet as GoogleTabNet

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2
FEAT_DIM = 5
ATTN_DIM = 2
OUTPUT_DIM = 1
OUTPUT_ACTIVATION = None

tf.random.set_seed(1)
model = LocalTabNet(
    FEAT_DIM,
    ATTN_DIM,
    OUTPUT_DIM,
    OUTPUT_ACTIVATION,
    sparsity=1
)

2023-06-06 14:23:07.883732: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-06-06 14:23:07.883790: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (al3615): /proc/driver/nvidia/version does not exist
2023-06-06 14:23:07.884260: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
dummy_in = tf.ones((1, FEAT_DIM), dtype="float")
model.compile(
    optimizer="adam",
    loss="mse",
    metrics=["mae", "mse"]
)
model.predict(dummy_in)

array([[-0.1533266]], dtype=float32)

In [8]:
model.metrics_names

['mask_entropy']

# Run a step of fit and validate

In [9]:
dummy_dataset = tf.data.Dataset.from_tensor_slices(
    (
        tf.ones((1, FEAT_DIM), dtype="float"),
        tf.ones((1, OUTPUT_DIM), dtype="float")
    )
).batch(1)
model.fit(
    dummy_dataset,
    epochs=1,
    batch_size=1,
    verbose=1,
    validation_data=dummy_dataset,
    validation_freq=1,
)

model.metrics_names

1/1 [==============================] - 10s 10s/step - loss: 2.6074 - mae: 0.9990 - mse: 0.9980 - mask_entropy: 1.6094 - val_loss: 2.9764 - val_mae: 1.1874 - val_mse: 1.4100 - val_mask_entropy: 1.5664


['loss', 'mae', 'mse', 'mask_entropy']

# Compare to google implementation
Currently not working

In [ ]:
%autoreload 2
tf.random.set_seed(1)
model = GoogleTabNet(
    columns=[tf.feature_column.numeric_column(str(ci)) for ci in range(FEAT_DIM)],
    num_features=FEAT_DIM,
    feature_dim=ATTN_DIM,
    output_dim=ATTN_DIM,
    num_decision_steps=5,
    relaxation_factor=1.5,
    batch_momentum=0.95,
    virtual_batch_size=None,
    num_classes=OUTPUT_DIM
)
decision, total_entropy = model.encoder(dummy_in, reuse=False, is_training=False)
out = model.regress(decision, reuse=False)